In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [16]:
#df_1 parecería no tener problemas con sus caracteres, df_2 si. 
df_1 = pd.read_csv('cine_nacional_trabajada.csv', error_bad_lines=False)
df_2 = pd.read_csv('cine_nacional_trabajada_aj_man.csv', encoding= 'latin1', error_bad_lines=False)

Skipping line 3: expected 4 fields, saw 6
Skipping line 15: expected 4 fields, saw 5
Skipping line 22: expected 4 fields, saw 5
Skipping line 43: expected 4 fields, saw 5
Skipping line 58: expected 4 fields, saw 6
Skipping line 113: expected 4 fields, saw 5
Skipping line 118: expected 4 fields, saw 6
Skipping line 279: expected 4 fields, saw 5
Skipping line 330: expected 4 fields, saw 6
Skipping line 402: expected 4 fields, saw 5
Skipping line 472: expected 4 fields, saw 5
Skipping line 478: expected 4 fields, saw 7
Skipping line 518: expected 4 fields, saw 5
Skipping line 559: expected 4 fields, saw 5
Skipping line 599: expected 4 fields, saw 7
Skipping line 639: expected 4 fields, saw 12
Skipping line 680: expected 4 fields, saw 5
Skipping line 681: expected 4 fields, saw 7
Skipping line 727: expected 4 fields, saw 6
Skipping line 743: expected 4 fields, saw 8
Skipping line 770: expected 4 fields, saw 6
Skipping line 833: expected 4 fields, saw 7
Skipping line 846: expected 4 fields,

In [40]:
#Sacamos 18 datos sin sinopsis
df_1.head()

df_1.columns = ['ID', 'Titulo','Anio de Produccion', 'Sinopsis']

df_1 = df_1[df_1['Sinopsis'].notnull()]


In [41]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

sinopsis = df_1['Sinopsis']

no_features = 1000

# NMF is able to use tf-idf
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, max_features=no_features,)
tfidf = tfidf_vectorizer.fit_transform(sinopsis)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

# LDA can only use raw term counts for LDA because it is a probabilistic graphical model
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=no_features)
tf = tf_vectorizer.fit_transform(sinopsis)
tf_feature_names = tf_vectorizer.get_feature_names()

In [42]:
from sklearn.decomposition import NMF, LatentDirichletAllocation

no_topics = 10

# Run NMF
nmf = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)

# Run LDA
lda = LatentDirichletAllocation(n_topics=no_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(tf)

In [45]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print "Topic %d:" % (topic_idx)
        print " ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]])

no_top_words = 10
print 'Modelo NMF:'
display_topics(nmf, tfidf_feature_names, no_top_words)
print '\nModelo LDA:'
display_topics(lda, tf_feature_names, no_top_words)

Modelo NMF:
Topic 0:
de la en las sus argentina por militar dictadura durante
Topic 1:
documental sobre ficción inmigrantes mujeres perón raúl argentina estas mario
Topic 2:
una de se en que comedia chica mujer donde amor
Topic 3:
el en por con las film se de es que
Topic 4:
su con que de por lo en es al padre
Topic 5:
la que historia es cuenta entre película narra no esta
Topic 6:
aires buenos en las pueblo ciudad calles semana económica festival
Topic 7:
un de se para que hombre en con pueblo grupo
Topic 8:
los en entre todos son años por de se uno
Topic 9:
del en vida secuestro parte rosario la siglo película br

Modelo LDA:
Topic 0:
distinto meses relata cuento maestro escuela un les el de
Topic 1:
de en un que la su los una el che
Topic 2:
dictadura militar de última la durante en desaparecido un documental
Topic 3:
cantor reencuentra exitoso natal viejos vuelve grande tierra amigos un
Topic 4:
de la su en una que por con el es
Topic 5:
la en su de el los del que por sus
Topic 6:
